# OCR for Handwritten Forms

In [21]:
import easyocr

In [ ]:
# Apply OCR to extract handwritten text

# Initialize reader object
reader = easyocr.Reader(['en'])

# Extract text from an image (replace 'image_path' with your image file)
result = reader.readtext('images\image_1.jpg', detail=0)

# Join the extracted text into a single string
extracted_text = '\n'.join(result)

print(extracted_text)  # Check the output

In [ ]:
# Save the extracted text to a file
with open('output_text.txt', 'w') as file:
    file.write(extracted_text)

print("Text saved to output_text.txt")

In [ ]:
import spacy

# Load the English model
nlp = spacy.load('en_core_web_sm')

# Read the saved text file
with open('output_text.txt', 'r') as file:
    text = file.read()

# Tokenize using spaCy
doc = nlp(text)
tokens = [token.text for token in doc]

print(tokens)